# ОН ДАЖЕ С ФИЛЬТРАМИ НЕ СЧИТАЕТ

In [2]:
import os
import csv
import wfdb
import heartpy as hp
import pandas as pd
import numpy as np
import scipy.signal as ssignal

In [3]:
age_group = {15:1,16:1,17:1,18:1,19:1,
             20:2,21:2,22:2,23:2,24:2,
             25:3,26:3,27:3,28:3,29:3,
             30:4,31:4,32:4,33:4,34:4,
             35:5,36:5,37:5,38:5,39:5,
             40:6,41:6,42:6,43:6,44:6,
             45:7,46:7,47:7,48:7,49:7,
             50:8,51:8,52:8,53:8,54:8,
             55:9,56:9,57:9,58:9,59:9,
             60:10,61:10,62:10,63:10,64:10,
             65:11,66:11,67:11,68:11,69:11,
             70:12,71:12,72:12,73:12,74:12,
             75:13,76:13,77:13,78:13,79:13,
             80:14,81:14,82:14,83:14,84:14,
             85:15,86:15,87:15,88:15,89:15,
             90:16,91:16,92:16,93:16,94:16,
             95:17,96:17,97:17,98:17,99:17,100:17,
             }

In [4]:
subjects = pd.read_excel(r"D:\Proga\AML\datasets\lucilacapurro\PPG_Signal_Dataset\subjects_metadata.xlsx",dtype = {"ID":str})
#subjects.dropna(inplace=True)
subjects = subjects[subjects['observations'].isna()]
subjects = subjects.reset_index()
subjects = subjects[["subjectcode","age"]]
subjects["age"] = subjects["age"].map(age_group)
subjects["age"]

0      1
1      1
2      1
3      1
4      1
5      1
6      1
7      1
8      1
9      1
10     2
11     2
12     2
13     2
14     2
15     2
16     2
17     2
18     2
19     2
20     2
21     2
22     2
23     2
24     2
25     2
26     2
27     2
28     2
29     2
30     2
31     2
32     2
33     3
34     4
35     4
36     4
37     6
38     6
39     6
40     7
41     7
42     7
43     8
44     8
45     8
46     8
47     9
48     9
49    14
50    15
Name: age, dtype: int64

In [5]:
record = wfdb.rdrecord('D:\\Proga\\AML\datasets\\physionet.org\\files\\pulse-transit-time-ppg\\1.1.0\\s1_sit')
print(record.__dict__)

{'record_name': 's1_sit', 'n_sig': 18, 'fs': 500, 'counter_freq': None, 'base_counter': None, 'sig_len': 254026, 'base_time': None, 'base_date': None, 'comments': ['<filename>: s1_sit <activity>: sit <gender>: female <height>: 160 <weight>: 50 <age>: 25 <bp_sys_start>: 87 <bp_sys_end>: 87 <bp_dia_start>: 59 <bp_dia_end>: 57 <hr_1_start>: 74 <hr_2_start>: 69 <hr_1_end>: 74 <hr_2_end>: 70 <spo2_start>: 98 <spo2_end>: 98'], 'sig_name': ['ecg', 'pleth_1', 'pleth_2', 'pleth_3', 'pleth_4', 'pleth_5', 'pleth_6', 'lc_1', 'lc_2', 'temp_1', 'temp_2', 'temp_3', 'a_x', 'a_y', 'a_z', 'g_x', 'g_y', 'g_z'], 'p_signal': array([[ 3.07405002e+04,  7.16949830e+04,  7.37368418e+04, ...,
         7.73396744e-03, -4.82016791e-04,  4.57538728e-03],
       [ 3.07200475e+04,  7.16798682e+04,  7.37258545e+04, ...,
         6.81947805e-03,  1.24659515e-03,  4.31670814e-03],
       [ 3.07098211e+04,  7.16798682e+04,  7.37258545e+04, ...,
         6.14014307e-03,  1.91144590e-03,  3.37899626e-03],
       ...,
    

In [6]:
def cheby(ppg, fs, order=4, rs=20, btype='low'):
    nyquist = 0.5 * fs
    cutoff = 10 / nyquist
    rs, order = 20, 4
    b, a = ssignal.cheby2(N=order, rs=rs, Wn=cutoff, btype='low')
    return ssignal.filtfilt(b, a, ppg)

In [7]:
result = []
result_metadata = []

new_id = 0
ids = range(1,23)
for id in ids:
    id = str(id)
    new_id += 1 
    try:
        record = wfdb.rdrecord('D:\\Proga\\AML\datasets\\physionet.org\\files\\pulse-transit-time-ppg\\1.1.0\\'+"s"+id+"_sit")

        #print(record.__dict__)
        #print(record.p_signal.shape)
        #channel_index = record.sig_name.index('NIBP')
        signal = record.p_signal.astype(np.float32)
        fs = record.fs

        five_minute_samples = 60*5*fs

        signal = signal[:five_minute_samples]
        signal = cheby(signal,fs,order = 4)
        
        try:
            wd, m = hp.process(signal.flatten(), fs)
            idx = int(subjects[subjects["ID"] == id].index.to_list()[0])
            age_group = age_group[idx]
            
            result_metadata.append({
                'id': new_id,
                'age_group': age_group,
                **m
            })
            
            print((new_id, id, age_group, signal, m))

        except hp.exceptions.BadSignalWarning as e:
            result_metadata.append({
                'id': new_id,
                'age_group': None,
                'error': "BadSignalWarning"
            })
            print((new_id, id, None, None, None, "BadSignalWarning"))

    except FileNotFoundError as e:
        result_metadata.append({
            'id': new_id,
            'age_group': None,
            'error': "FileNotFound"
        })
        print((new_id, id, None, None, None, "FileNotFound"))

    print(f"Process end: {id}")

(1, '1', None, None, None, 'BadSignalWarning')
Process end: 1
(2, '2', None, None, None, 'BadSignalWarning')
Process end: 2
(3, '3', None, None, None, 'BadSignalWarning')
Process end: 3


c:\Users\toha2\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\toha2\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\toha2\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\ma\core.py:5317: RuntimeWarning: Mean of empty slice.
  result = super().mean(axis=axis, dtype=dtype, **kwargs)[()]
c:\Users\toha2\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3747: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


KeyError: 'ID'